In [443]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

In [444]:
# Lets bring in trained df for the 'SalePrice'
train_df = pd.read_csv('../project_2-master/data/train.csv')

In [445]:
# For the baseline score
test_df = pd.read_csv('../project_2-master/data/test.csv')

In [446]:
# Lets read in the cleaned trained df
merged_train = pd.read_csv('../project_2-master/data/merged_train.csv')
# Lets read in the cleaned test df
merged_test = pd.read_csv('../project_2-master/data/merged_test.csv')

In [447]:
merged_train.head(1)

,Id,Total Bsmt SF,1st Flr SF,Gr Liv Area,Garage Area,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,...,Bsmt Qual,Kitchen Qual,Year Built,Full Bath,TotRms AbvGrd,Garage Cars,Grla_tbsf,ovl_ext,ovl_area,gr_ga
0,109,725.0,725,1479,475.0,0,0,0,0,0,...,3,4,1976,2,6,2.0,1072275.0,24,8874,702525.0


In [448]:
merged_test.head(1)

,Id,Total Bsmt SF,1st Flr SF,Gr Liv Area,Garage Area,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,...,Bsmt Qual,Kitchen Qual,Year Built,Full Bath,TotRms AbvGrd,Garage Cars,Grla_tbsf,ovl_ext,ovl_area,gr_ga
0,2658,1020,908,1928,440,0,0,0,0,0,...,2,2,1910,2,9,1,1966560,18,11568,848320


## Investment Regression

In [449]:
def linear_regression(train, test, saleprice):

# SET 'Id' AS THE INDEX
    train.set_index('Id', inplace=True)
    test.set_index('Id', inplace=True)

# CREATING X AND y
    X = train[train.columns[1:]]
    y = saleprice['SalePrice']
    
# POLYNOMIAL FEATURE ENGINEERING
    # Instantiate our PolynomialFeatures object to create all two-way terms.
#     poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)

    # Fit and transform our X data.
#     X_overfit = poly.fit_transform(X[train.columns[1:7]])

# TRAIN, TEST, SPLIT
    # train, test, split
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, train_size = .90)
    
    #  Instantiate the model
    lr = LinearRegression()
    
    # Fit the model
    lr.fit(X_train, y_train)
    
    # Predict the model
    y_pred_train = lr.predict(X_train)
    y_pred_test = lr.predict(X_test)
    
    coefs = lr.coef_
    
    # Get the RMSE
    print(f"The RMSE for X_train is {mean_squared_error(y_train, y_pred_train, squared=False)}")
    print(f"The RMSE for X_test is {mean_squared_error(y_test, y_pred_test, squared=False)}")
    
# KAGGLE SUBMISSION STEPS

    # Set my merged_test to kaggle
    kaggle_predict = merged_test[merged_test.columns[1:]]
    
    # Predict 'SalePrice' with our model
    kaggle_predict['SalePrice'] = lr.predict(kaggle_predict)
    
#     resids = y - kaggle_predict['SalePrice']
    
    output = kaggle_predict[['SalePrice']]
    
    output.to_csv('../project_2-master/data/new_submission.csv')
    
    return resids.isna().sum()

In [450]:
linear_regression(merged_train, merged_test, train_df)

The RMSE for X_train is 29808.49718894459
The RMSE for X_test is 30517.328043451882


1701

## Ridge Regresssion

In [365]:
def ridge_regression(train, test, saleprice):

# SET 'Id' AS THE INDEX
    train.set_index('Id', inplace=True)
    test.set_index('Id', inplace=True)

# CREATING X AND y
    X = train[train.columns[1:]]
    y = saleprice['SalePrice']

# TRAIN, TEST, SPLIT
    # train, test, split
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, train_size = .90)
    
    
# STANDARD SCALER
    # Relabeling scaled data as "Z" is common.
    sc = StandardScaler()
    Z_train = sc.fit_transform(X_train)
    Z_test = sc.transform(X_test)


    # Instantiate.
    ridge_model = Ridge(alpha=10)

    # Fit.
    ridge_model.fit(Z_train, y_train)

    # Evaluate model using R2.
    z_tet= print(ridge_model.score(Z_train, y_train))
    z_tot = print(ridge_model.score(Z_test, y_test))


# KAGGLE SUBMISSION STEPS

    # Set my merged_test to kaggle
    kaggle_predict = merged_test[merged_test.columns[1:]]
    
    # Predict 'SalePrice' with our model
    kaggle_predict['SalePrice'] = ridge_model.predict(kaggle_predict)
    
    
    output = kaggle_predict[['SalePrice']]
    
    output.to_csv('../project_2-master/data/ridge_submission.csv')
    
    return output

In [366]:
ridge_regression(merged_train, merged_test, train_df)

0.8574369588674452
0.847941030564203


,SalePrice
Id,
2658,-3.657231e+10
2718,-7.251554e+10
2414,-1.798264e+10
1989,-1.744055e+10
625,-3.626117e+10
...,...
1662,-3.786665e+10
1234,-4.085717e+10
1373,-2.145691e+10


## Baseline Score

In [397]:
def baseline_score(train, test, saleprice):

# CREATING X AND y
    X = train[train.columns[1:]]
    y = saleprice['SalePrice']
    
    y_bar = np.mean(y)
    
    null_mse = np.mean((y - y_bar)**2)


    return null_mse

In [398]:
baseline_score(train_df, test_df, train_df)

6278872217.837828